In [1]:
#!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
import cv2
cv2.__version__

'4.1.2'

In [1]:
!nvidia-smi

Wed Sep 30 05:59:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# https://medium.com/analytics-vidhya/implementing-yolov4-to-detect-custom-objects-using-google-colab-6691c98b15ff

####  STEPS   ####
# clone the repo
# set GPU card config in makefile (line no. 27-50)
# install using make

# prepare the dataset & the annotations ( .txt file in the same directory with the same name. 
# Each text file contains the class id and coordinates of a bounding box for every object as shown below
# <object-class> <x_center> <y_center> <width> <height> 
# <x_center> <y_center> <width> <height> are float values relative to width and height of image,
# it can be equal from (0.0 to 1.0])
# if the annotations in the source data are not in yolo format, convert to that format using https://github.com/ssaru/convert2Yolo

# create configuration file (.cfg) & set model parameters here
# copy yolo4-custom.cfg now - rename the copied file to yolo-obj.cfg

# create the following files describing the data
# obj.names, obj.data, train.txt, validate.txt, test.txt

# download the pre-trained weights "yolov4.conv.137" for the convolutional layers and put in the directory build\darknet\x64

# After training is complete weights will be saved as yolo-obj_last.weights for every 100 iterations and 
# saved as yolo-obj_xxxx.weights for every 1000 iterations in the directory build\darknet\x64\backup

# test predictions

# saving intermediate checkpoints
# https://medium.com/ai-world/how-to-train-yolov4-for-custom-objects-detection-in-google-colab-1e934b8ef685
# intermediate weights saved in folder: build/darknet/x64/backup/


### ISSUE WITH make command & training code - with GPU & CUDA enabled ###
# https://github.com/pjreddie/darknet/issues/200

# after removing space in makefile for K80 config, make command runs but training still gives same error

CUDA status Error: file: ./src/blas_kernels.cu : () : line: 859 : build time: Aug 31 2020 - 13:55:36 

 CUDA Error: no kernel image is available for execution on the device
CUDA Error: no kernel image is available for execution on the device: File exists
darknet: ./src/utils.c:326: error: Assertion `0' failed.

# upon checking, I found change in assignment from K80 to T4 - colab GPU changes?
# correcting the config runs the code

# alternate option: compile only with openCV & run

In [ ]:
# to avoid colab disconnection
press ctrl+shift+i simultaneously. This will open console and in the console paste the following code.

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)

In [2]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''# extract the zip file - only 1st time
!unzip -q "/content/drive/My Drive/object_detection/darknet.zip" -d "/content/gdrive/My Drive/object_detection/"
'''

In [ ]:
# copy to root
#import os
#os.chdir("/content/")
#!cp -r /content/drive/My Drive/object_detection/darknet /content/

In [3]:
import os
os.chdir('/content/drive/My Drive/VQI/darknet')

In [ ]:
!ls

3rdparty		       DarknetConfig.cmake.in  obj
backup			       darknet_images.py       prediction_img.jpg
bad.list		       darknet.py	       README.md
build			       darknet_video.py        results
build.ps1		       data		       result.txt
build.sh		       image_yolov3.sh	       scripts
cfg			       image_yolov4.sh	       src
chart_imp-tiny-yolo4_v3_5.png  include		       test2.jpg
chart_imp-tiny-yolo4_v3.png    json_mjpeg_streams.sh   test.jpg
chart.png		       LICENSE		       video_yolov3.sh
cmake			       Makefile.txt	       video_yolov4.sh
CMakeLists.txt		       net_cam_v3.sh	       z_sree_run_yolov4
darknet			       net_cam_v4.sh


In [4]:
!cat Makefile

GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)
# set ZED_CAMERA=1 to enable ZED SDK 3.0 and above
# set ZED_CAMERA_v2_8=1 to enable ZED SDK 2.X

USE_CPP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	    -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla A100 (GA100), DGX-A100, RTX 3080
# ARCH= -gencode arch=compute_80,code=[sm_80,compute_80]

#Tesla K80
#ARCH= -gencode arch=compute_37,code=sm_37

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro 

In [5]:
!sed -i 's/#ARCH= -gencode arch=compute_37,code=sm_37/ARCH= -gencode arch=compute_37,code=sm_37/g' Makefile
!sed -i 's/ARCH= -gencode arch=compute_75,/#ARCH= -gencode arch=compute_75,/g' Makefile

In [6]:
!cat Makefile

GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)
# set ZED_CAMERA=1 to enable ZED SDK 3.0 and above
# set ZED_CAMERA_v2_8=1 to enable ZED SDK 2.X

USE_CPP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	    -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla A100 (GA100), DGX-A100, RTX 3080
# ARCH= -gencode arch=compute_80,code=[sm_80,compute_80]

#Tesla K80
ARCH= -gencode arch=compute_37,code=sm_37

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro R

In [7]:
!sudo chmod +x darknet 

In [ ]:
#!ln -s /usr/local/cuda /usr/local/cuda-10.1/
#!export PATH=/usr/local/cuda/bin${PATH:+:${PATH}}

In [ ]:
#!set NVCC = /usr/local/cuda/bin/nvcc

In [8]:
#!sudo make
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function â€˜void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)â€™:
./src/image_opencv.cpp:926:23: warning: variable â€˜rgbâ€™ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function â€˜void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)â€™:
./src/image_opencv.cpp:1127:13: warning: this â€˜ifâ€™ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130:

In [ ]:
'''os.chdir('/content/drive/My Drive/VQI/')
os.mkdir('darknet2')
os.chdir('darknet2')

In [ ]:
'''!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 14285, done.
remote: Total 14285 (delta 0), reused 0 (delta 0), pack-reused 14285
Receiving objects: 100% (14285/14285), 12.83 MiB | 5.46 MiB/s, done.
Resolving deltas: 100% (9747/9747), done.
Checking out files: 100% (2014/2014), done.


In [ ]:
#os.chdir('darknet')
'''!make  # version without GPU specified in makefile

chmod +x *.sh


In [ ]:
###  starting training the 1st time  ###

! ./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolov4.conv.137 -dont_show -mjpeg_port 8090 -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 56613, rewritten_bbox = 0.181937 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.376408, GIOU: 0.376183), Class: 0.514672, Obj: 0.008669, No Obj: 0.008598, .5R: 0.000000, .75R: 0.000000, count: 4, class_loss = 3.633216, iou_loss = 0.916682, total_loss = 4.549898 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.370426, GIOU: 0.290520), Class: 0.573247, Obj: 0.022696, No Obj: 0.020807, .5R: 0.250000, .75R: 0.000000, count: 24, class_loss = 16.989058, iou_loss = 1.325348, total_loss = 18.314405 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.260830, GIOU: 0.172927), Class: 0.516913, Obj: 0.066065, No Obj: 0.058919, .5R: 0.083333, .75R: 0.000000, count: 12, class_loss = 9.122010, iou_loss = 0.083456, total_loss = 9.205466 
 total_bbox = 56653, rewritten_bbox = 0.181809 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.333743, 

<IPython.core.display.Javascript object>

In [ ]:
###  continuing training from the last saved weights : yolo-obj_xxxx.weights  ###

#! ./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_2000.weights -dont_show -mjpeg_port 8090 -map
! ./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_last.weights -dont_show -mjpeg_port 8090 -map

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.753319, GIOU: 0.739647), Class: 0.994463, Obj: 0.488360, No Obj: 0.002003, .5R: 0.958333, .75R: 0.500000, count: 24, class_loss = 3.565325, iou_loss = 9.699927, total_loss = 13.265253 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.745169, GIOU: 0.739682), Class: 0.995105, Obj: 0.472034, No Obj: 0.007571, .5R: 1.000000, .75R: 0.600000, count: 25, class_loss = 2.798410, iou_loss = 2.639004, total_loss = 5.437414 
 total_bbox = 86121, rewritten_bbox = 0.207847 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.819855, GIOU: 0.811765), Class: 0.992759, Obj: 0.031532, No Obj: 0.000010, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.235262, iou_loss = 1.911836, total_loss = 2.147098 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.694348, GIOU: 0.686058), Class: 0.983667, Obj: 0.180034, No

In [ ]:
!pwd

/content/drive/My Drive/VQI/darknet


In [ ]:
# testing the algorithm
# yolo-obj_final.weights: saves weights once training is done

#!./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_final.weights test.jpg -dont_show
!./darknet detector test ./build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights test.jpg -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rou

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights test.jpg

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rou

In [ ]:
!echo 'test.jpg' > infer_image_list.txt
!cat infer_image_list.txt

test.jpg


In [ ]:
%%bash
cat > infer_image_list.txt << EOF
test.jpg
test2.jpg
EOF

In [ ]:
!cat infer_image_list.txt

test.jpg
test2.jpg


In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -dont_show -ext_output < ./infer_image_list.txt 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rou

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -dont_show -ext_output < ./infer_image_list.txt > pred_coord.txt

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv     64       

In [ ]:
!cat pred_coord.txt

net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 364 K-images (5 Kilo-batches_64) 
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
test.jpg: Predicted in 98.386000 milli-seconds.
scratches: 87%	(left_x:   -7   top_y:   -1   width:  213   height:   20)
scratches: 97%	(left_x:   -3   top_y:  140   width:   87   height:   15)
scratches: 99%	(left_x:   21   top_y:   98   width:  119   height:   29)
scratches: 95%	(left_x:  111   top_y:  140   width:   90   height:   16)
scratches: 96%	(left_x:  146   top_y:   24   width:   55   height:   29)
Enter Image Path: 

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -dont_show -save_labels < ./infer_image_list.txt >pred_coord.txt

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv     64       

In [ ]:
!cat pred_coord.txt

net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 364 K-images (5 Kilo-batches_64) 
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
test.jpg: Predicted in 97.998000 milli-seconds.
scratches: 87%
scratches: 97%
scratches: 99%
scratches: 95%
scratches: 96%
Enter Image Path: 

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -ext_output -dont_show -out result.json < ./infer_image_list.txt 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rou

In [ ]:
!cat result.json

[
{
 "frame_id":1, 
 "filename":"test.jpg", 
 "objects": [ 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.402204, "center_y":0.560369, "width":0.596843, "height":0.144730}, "confidence":0.993152}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.200272, "center_y":0.735696, "width":0.434813, "height":0.073243}, "confidence":0.971898}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.868423, "center_y":0.192251, "width":0.275006, "height":0.146317}, "confidence":0.960154}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.781022, "center_y":0.738446, "width":0.449532, "height":0.079440}, "confidence":0.948475}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.497478, "center_y":0.045353, "width":1.067175, "height":0.101833}, "confidence":0.869762}
 ] 
}
]

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -ext_output -dont_show -out pred_coord.txt < ./infer_image_list.txt 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rou

In [ ]:
!cat pred_coord.txt

[
{
 "frame_id":1, 
 "filename":"test.jpg", 
 "objects": [ 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.402204, "center_y":0.560369, "width":0.596843, "height":0.144730}, "confidence":0.993152}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.200272, "center_y":0.735696, "width":0.434813, "height":0.073243}, "confidence":0.971898}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.868423, "center_y":0.192251, "width":0.275006, "height":0.146317}, "confidence":0.960154}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.781022, "center_y":0.738446, "width":0.449532, "height":0.079440}, "confidence":0.948475}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.497478, "center_y":0.045353, "width":1.067175, "height":0.101833}, "confidence":0.869762}
 ] 
}
]

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -dont_show -ext_output < ./infer_image_list.txt > pred_coord2.txt

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv     64       1

In [ ]:
!cat pred_coord2.txt

net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 364 K-images (5 Kilo-batches_64) 
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
test.jpg: Predicted in 43.427000 milli-seconds.
scratches: 87%	(left_x:   -7   top_y:   -1   width:  213   height:   20)
scratches: 97%	(left_x:   -3   top_y:  140   width:   87   height:   15)
scratches: 99%	(left_x:   21   top_y:   98   width:  119   height:   29)
scratches: 95%	(left_x:  111   top_y:  140   width:   90   height:   16)
scratches: 96%	(left_x:  146   top_y:   24   width:   55   height:   29)
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
test2.jpg: Predicted in 43.386000 milli-seconds.
patches: 100%	(left_x:   47   to

In [ ]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -ext_output -dont_show -out result2.json < ./infer_image_list.txt 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rout

In [ ]:
!cat result2.json

[
{
 "frame_id":1, 
 "filename":"test.jpg", 
 "objects": [ 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.402204, "center_y":0.560369, "width":0.596843, "height":0.144730}, "confidence":0.993152}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.200272, "center_y":0.735696, "width":0.434813, "height":0.073243}, "confidence":0.971898}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.868423, "center_y":0.192251, "width":0.275006, "height":0.146317}, "confidence":0.960154}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.781022, "center_y":0.738446, "width":0.449532, "height":0.079440}, "confidence":0.948475}, 
  {"class_id":5, "name":"scratches", "relative_coordinates":{"center_x":0.497478, "center_y":0.045353, "width":1.067175, "height":0.101833}, "confidence":0.869763}
 ] 
}, 
{
 "frame_id":2, 
 "filename":"test2.jpg", 
 "objects": [ 
  {"class_id":2, "name":"patches", "relative_

In [9]:
!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg \
  ./build/darknet/x64/backup/yolov4_60_20_20/yolo-obj_last.weights \
  -dont_show < ./build/darknet/x64/data/test.txt > pred_test_coord2.txt

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv     64       

In [10]:
!cat pred_test_coord2.txt

net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 364 K-images (5 Kilo-batches_64) 
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
/content/drive/My Drive/VQI/darknet/build/darknet/x64/data/obj/Combined/rolled-in_scale_237.jpg: Predicted in 101.573000 milli-seconds.
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
/content/drive/My Drive/VQI/darknet/build/darknet/x64/data/obj/Combined/inclusion_259.jpg: Predicted in 97.844000 milli-seconds.
inclusion: 99%
Enter Image Path: 

In [ ]:
# yolo-obj_best.weights: saves weights for highest mAP during training

!./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_best.weights test.jpg -dont_show -benchmark 